In [1]:
import os, cv2
import numpy as np
import tensorflow as tf

In [2]:
gpus=tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
     # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu,True)
        logical_gpus=tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus),"Physical GPUs,",len(logical_gpus),"Logical GPUs")
    except RuntimeError as e:
        #Memory growth must be set before GPUs have been initialized
        print(e)

1 Physical GPUs, 1 Logical GPUs


In [3]:
TRAIN_DIR = 'C:/Users/Arthur_Noh/Desktop/dataset/Train'
TEST_DIR = 'C:/Users/Arthur_Noh/Desktop/dataset/Test'

CATEGORIES = ['blackice','road','snow','wet']

IMG_SIZE = 150

In [4]:
X_train = []
Y_train = []

X_test = []
Y_test = []

In [5]:
# training data
for idx, category in enumerate(CATEGORIES):
    # one-hot incoding
    label = [0 for i in range(len(CATEGORIES))]
    label[idx] = 1
    
    path = os.path.join(TRAIN_DIR, category)
    # check to see if read images work
    print(category, '의 파일 길이 : ', len(os.listdir(path)))
    
    for i, image in enumerate(os.listdir(path)):
        img = cv2.imread(os.path.join(path, image),
                        cv2.IMREAD_GRAYSCALE)        
        # Make image size that we want
        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
        data = np.array(img)
        
        # input data in temp variable
        X_train.append(data)
        Y_train.append(label)
        
        # check image lists
        if i % 2000 == 0:
            print(category, ':', image)
    print('\n')

blackice 의 파일 길이 :  10000
blackice : b_1.jpg
blackice : b_2799.jpg
blackice : b_4599.jpg
blackice : b_6399.jpg
blackice : b_8199.jpg


road 의 파일 길이 :  10000
road : r_1.jpg
road : r_2799.jpg
road : r_4599.jpg
road : r_6399.jpg
road : r_8199.jpg


snow 의 파일 길이 :  10000
snow : s_1.jpg
snow : s_2799.jpg
snow : s_4599.jpg
snow : s_6399.jpg
snow : s_8199.jpg


wet 의 파일 길이 :  10000
wet : w_1.jpg
wet : w_2799.jpg
wet : w_4599.jpg
wet : w_6399.jpg
wet : w_8199.jpg




In [6]:
# testing data
for idx, category in enumerate(CATEGORIES):
    # one-hot incoding
    label = [0 for i in range(len(CATEGORIES))]
    label[idx] = 1
    
    path = os.path.join(TEST_DIR, category)
    # check to see if read images work
    print(category, '의 파일 길이 : ', len(os.listdir(path)))
    
    for i, image in enumerate(os.listdir(path)):
        img = cv2.imread(os.path.join(path, image),
                        cv2.IMREAD_GRAYSCALE)        
        # Make image size that we want
        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
        data = np.array(img)
        
        # input data in temp variable
        X_test.append(data)
        Y_test.append(label)
        
        # check image lists
        if i % 200 == 0:
            print(category, ':', image)
    print('\n')

blackice 의 파일 길이 :  1000
blackice : b_2901.jpg
blackice : b_3101.jpg
blackice : b_3301.jpg
blackice : b_3501.jpg
blackice : b_3701.jpg


road 의 파일 길이 :  1000
road : r_3901.jpg
road : r_4101.jpg
road : r_4301.jpg
road : r_4501.jpg
road : r_4701.jpg


snow 의 파일 길이 :  1000
snow : s_2901.jpg
snow : s_3101.jpg
snow : s_3301.jpg
snow : s_3501.jpg
snow : s_3701.jpg


wet 의 파일 길이 :  1000
wet : w_3901.jpg
wet : w_4101.jpg
wet : w_4301.jpg
wet : w_4501.jpg
wet : w_4701.jpg




In [7]:
X_train = np.array(X_train).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
X_test = np.array(X_test).reshape(-1, IMG_SIZE, IMG_SIZE, 1)

Y_train = np.array(Y_train)
Y_test = np.array(Y_test)

In [8]:
print(X_train.shape, Y_train.shape)
print(X_test.shape, Y_test.shape)

(40000, 150, 150, 1) (40000, 4)
(4000, 150, 150, 1) (4000, 4)


In [ ]:
print(Y_train)

In [9]:
from tensorflow import keras
model=tf.keras.Sequential([
    tf.keras.layers.Conv2D(input_shape=(150,150,1), kernel_size=(3,3), filters=16, padding='same',activation='relu'),
    tf.keras.layers.Conv2D(input_shape=(150,150,1), kernel_size=(3,3), filters=32, padding='same',activation='relu'),
    tf.keras.layers.MaxPool2D(strides=(2,2)),
    tf.keras.layers.MaxPool2D(strides=(2,2)),
    tf.keras.layers.Dropout(rate=0.2),
    
    tf.keras.layers.Conv2D(kernel_size=(3,3), filters=64, padding='same', activation='relu'),
    tf.keras.layers.Conv2D(kernel_size=(3,3), filters=128, padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D(strides=(2,2)),
    tf.keras.layers.MaxPool2D(strides=(2,2)),
    tf.keras.layers.Dropout(rate=0.2),
    
    tf.keras.layers.Conv2D(kernel_size=(3,3), filters=256, padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D(strides=(2,2)),
    tf.keras.layers.Dropout(rate=0.2),
    
    tf.keras.layers.Conv2D(kernel_size=(3,3), filters=256, padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D(strides=(2,2)),
    tf.keras.layers.Dropout(rate=0.2),
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=1024, activation='relu'),
    tf.keras.layers.Dropout(rate=0.2),
    tf.keras.layers.Dense(units=512, activation='relu'),
    tf.keras.layers.Dropout(rate=0.2),
    tf.keras.layers.Dense(units=256, activation='relu'),
    tf.keras.layers.Dropout(rate=0.2),
    tf.keras.layers.Dense(units=4, activation='softmax')
])
# sparse 빼도됨
model.compile(optimizer=tf.keras.optimizers.Adam(),
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 150, 150, 16)      160       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 150, 150, 32)      4640      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 75, 75, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 37, 37, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 37, 37, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 37, 37, 64)        18496     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 37, 37, 128)       7

In [21]:
from keras.layers import Conv2D, MaxPool2D, Dropout, Flatten, Dense

model = tf.keras.Sequential()

model.add(Conv2D(filters=16, kernel_size=(3,3),
                padding='same', input_shape=(150, 150, 1),
                activation='relu'))
model.add(Conv2D(filters=32, kernel_size=(3,3),
                padding='same', activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(rate=0.5))


model.add(Conv2D(filters=64, kernel_size=(3,3),
                padding='same', activation='relu'))
model.add(Conv2D(filters=128, kernel_size=(3,3),
                padding='same', activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(rate=0.5))

model.add(Conv2D(filters=256, kernel_size=(3,3),
                padding='same', activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(rate=0.5))

model.add(Conv2D(filters=256, kernel_size=(3,3),
                padding='same', activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(rate=0.5))

model.add(Flatten())
model.add(Dense(units=1024, activation='relu'))
model.add(Dropout(rate=0.25))
model.add(Dense(units=512, activation='relu'))
model.add(Dropout(rate=0.25))
model.add(Dense(units=256, activation='relu'))
model.add(Dropout(rate=0.25))
model.add(Dense(units=4, activation='softmax'))

In [22]:
model.compile(
    optimizer = tf.keras.optimizers.Adam(),
    loss = 'categorical_crossentropy',
    metrics = ['accuracy']
)

In [23]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_18 (Conv2D)           (None, 150, 150, 16)      160       
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 150, 150, 32)      4640      
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 75, 75, 32)        0         
_________________________________________________________________
max_pooling2d_17 (MaxPooling (None, 37, 37, 32)        0         
_________________________________________________________________
dropout_21 (Dropout)         (None, 37, 37, 32)        0         
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 37, 37, 64)        18496     
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 37, 37, 128)      

In [ ]:
from keras.callbacks import EarlyStopping
early_stopping=EarlyStopping(monitor='val_loss', mode='min', min_delta=1e-3, patience=20, verbose=1) 
history=model.fit(X_train, Y_train, epochs=100, validation_split=0.2, callbacks=[early_stopping])

Epoch 1/100
1000/1000 [==============================] - 21s 21ms/step - loss: 1.0082 - accuracy: 0.6406 - val_loss: 1.6917 - val_accuracy: 0.0000e+00
Epoch 2/100
1000/1000 [==============================] - 21s 21ms/step - loss: 0.6490 - accuracy: 0.7270 - val_loss: 1.5193 - val_accuracy: 0.2705
Epoch 3/100
1000/1000 [==============================] - 21s 21ms/step - loss: 0.5671 - accuracy: 0.7770 - val_loss: 1.9620 - val_accuracy: 0.0626
Epoch 4/100
1000/1000 [==============================] - 21s 21ms/step - loss: 0.4775 - accuracy: 0.8270 - val_loss: 1.8576 - val_accuracy: 0.0000e+00
Epoch 5/100
1000/1000 [==============================] - 21s 21ms/step - loss: 0.4483 - accuracy: 0.8369 - val_loss: 1.7750 - val_accuracy: 0.0000e+00
Epoch 6/100
1000/1000 [==============================] - 21s 21ms/step - loss: 0.4043 - accuracy: 0.8537 - val_loss: 2.3861 - val_accuracy: 0.1215
Epoch 7/100
1000/1000 [==============================] - 21s 21ms/step - loss: 0.3981 - accuracy: 0.8563 -

In [ ]:
import matplotlib.pyploy as plt

plt.figure(figsize=(12,4))

plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], 'b-', label='loss')
plt.plot(history.history['val_loss'], 'r--', label='val_loss')
plt.xlabel('Epoch')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], 'g-', label='accuracy')
plt.plot(history.history['val_accuracy'], 'k--', label='val_accuracy')
plt.xlabel('Epoch')

plt.legend()

plt.show()